In [1]:
import os
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, UpSampling2D, Conv2D, Conv2DTranspose, Reshape, Input
from tensorflow.keras import optimizers
from efficientnet.tfkeras import EfficientNetB2, preprocess_input
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import cv2
import MVTecAD_loaddata as mv
import pandas as pd
res = []
def OCSVM_split(X, y, train_data):
    tuned_parameters = [
    {'nu': [2**-3, 2**-4, 2**-5,2**-6,2**-7], 'kernel': ['rbf'], 'gamma': [10**-3, 10**-4, 10**-5, 10**-6, 10**-7]},
    ]
    score = 'accuracy'
    clf = GridSearchCV(
        OneClassSVM(),
        tuned_parameters,
        cv=5,
        scoring=score)
#         scoring='%s_weighted' % score )
    clf.fit(X, y)
    svm = OneClassSVM(nu=clf.best_params_['nu'], gamma=clf.best_params_['gamma'])
    svm.fit(X,y)
    sv_num = svm.support_
    sv = train_data[sv_num,:]
    return clf, svm, sv

def OCSVM2(clf, svm, X, x_test, y, y_test):
    svm.fit(X, y)
    a = svm.decision_function(x_test)
    b = svm.predict(x_test)
    r1 = roc_auc_score(y_test,a)
    r2 = f1_score(y_test,b)
    r3 = accuracy_score(y_test,b)
    print('roc_auc : ',r1)
    print('f1_score : ', r2)
    print('accuracy_score : ', r3)
    print('best_params : ', clf.best_params_)
    result = [r1,r2,r3]
    return result

def model_fit(X_train, X_test, SVs, model):
    model.fit(SVs, SVs, epochs=20, batch_size=16, shuffle=True)
    encoder = Model(model.layers[0].input, model.layers[-8].output)
    f_tr = encoder.predict(X_train, verbose=0)
    f_ts = encoder.predict(X_test, verbose=0)
    x_tr = np.reshape(f_tr, (f_tr.shape[0], -1))
    x_ts = np.reshape(f_ts, (f_ts.shape[0], -1))
    
    incov_train = np.linalg.inv(np.cov(x_tr.T))
    incov_test = np.linalg.inv(np.cov(x_ts.T))
    
    x_tr = (x_tr@incov_train)
    x_ts = (x_ts@incov_test)
    
    x_tr = x_tr/(np.max(x_tr))
    x_ts = x_ts/(np.max(x_ts))
    return x_tr, x_ts

def model_predict(X_train):
    input_shape = (256, 256, 3)
    inputs = Input(input_shape)

    base_model = EfficientNetB2(include_top=False, weights='imagenet', pooling='avg')(inputs)
    x = Reshape((8, 8, 22))(base_model)
    x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
    outputs = Conv2D(3, kernel_size=(3, 3), activation='sigmoid', padding='same')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error")
    
    encoder = Model(model.layers[0].input, model.layers[-8].output)
    f_tr = encoder.predict(X_train, verbose=0)
    x_tr = np.reshape(f_tr, (f_tr.shape[0], -1))
    incov_train = np.linalg.inv(np.cov(x_tr.T))
    x_tr = (x_tr@incov_train)
    x_tr = x_tr/(np.max(x_tr))
    
    return model, x_tr

In [2]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
2/2 [==============================] - 86s 7s/step - loss: 0.1323
Epoch 2/20
2/2 [==============================] - 16s 7s/step - loss: 0.1323
Epoch 3/20
2/2 [==============================] - 16s 7s/step - loss: 0.1322
Epoch 4/20
2/2 [==============================] - 18s 7s/step - loss: 0.1322
Epoch 5/20
2/2 [==============================] - 18s 7s/step - loss: 0.1322
Epoch 6/20
2/2 [==============================] - 16s 6s/step - loss: 0.1322
Epoch 7/20
2/2 [==============================] - 16s 7s/step - loss: 0.1322
Epoch 8/20
2/2 [==============================] - 16s 7s/step - loss: 0.1321
Epoch 9/20
2/2 [==============================] - 16s 7s/step - loss: 0.1321
Epoch 10/20
2/2 [==============================] - 15s 6s/step - loss: 0.1321
Epoch 11/20
2/2 [==============================] - 16s 6s/step - loss: 0.1321
Epoch 12/20
2/2 [==============================] - 16s 7s/step - loss: 0.1321
Epoch 13/20
2/2 [==============================] - 16s 6s/step - loss: 0.

In [3]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
2/2 [==============================] - 73s 8s/step - loss: 0.0547
Epoch 2/20
2/2 [==============================] - 19s 7s/step - loss: 0.0546
Epoch 3/20
2/2 [==============================] - 15s 7s/step - loss: 0.0545
Epoch 4/20
2/2 [==============================] - 15s 7s/step - loss: 0.0545
Epoch 5/20
2/2 [==============================] - 15s 6s/step - loss: 0.0544
Epoch 6/20
2/2 [==============================] - 15s 6s/step - loss: 0.0543
Epoch 7/20
2/2 [==============================] - 15s 6s/step - loss: 0.0542
Epoch 8/20
2/2 [==============================] - 15s 6s/step - loss: 0.0540
Epoch 9/20
2/2 [==============================] - 15s 6s/step - loss: 0.0539
Epoch 10/20
2/2 [==============================] - 15s 6s/step - loss: 0.0537
Epoch 11/20
2/2 [==============================] - 15s 7s/step - loss: 0.0535
Epoch 12/20
2/2 [==============================] - 15s 6s/step - loss: 0.0533
Epoch 13/20
2/2 [==============================] - 15s 6s/step - loss: 0.

In [4]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
2/2 [==============================] - 58s 7s/step - loss: 0.0922
Epoch 2/20
2/2 [==============================] - 17s 7s/step - loss: 0.0921
Epoch 3/20
2/2 [==============================] - 16s 7s/step - loss: 0.0920
Epoch 4/20
2/2 [==============================] - 15s 7s/step - loss: 0.0919
Epoch 5/20
2/2 [==============================] - 15s 7s/step - loss: 0.0917
Epoch 6/20
2/2 [==============================] - 15s 7s/step - loss: 0.0916
Epoch 7/20
2/2 [==============================] - 15s 7s/step - loss: 0.0914
Epoch 8/20
2/2 [==============================] - 15s 7s/step - loss: 0.0911
Epoch 9/20
2/2 [==============================] - 316s 308s/step - loss: 0.0908
Epoch 10/20
2/2 [==============================] - 22s 6s/step - loss: 0.0905
Epoch 11/20
2/2 [==============================] - 14s 5s/step - loss: 0.0901
Epoch 12/20
2/2 [==============================] - 12s 5s/step - loss: 0.0895
Epoch 13/20
2/2 [==============================] - 12s 5s/step - loss:

In [5]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
3/3 [==============================] - 58s 6s/step - loss: 0.0385
Epoch 2/20
3/3 [==============================] - 17s 5s/step - loss: 0.0384
Epoch 3/20
3/3 [==============================] - 15s 4s/step - loss: 0.0382
Epoch 4/20
3/3 [==============================] - 15s 4s/step - loss: 0.0380
Epoch 5/20
3/3 [==============================] - 15s 4s/step - loss: 0.0377
Epoch 6/20
3/3 [==============================] - 15s 4s/step - loss: 0.0374
Epoch 7/20
3/3 [==============================] - 14s 4s/step - loss: 0.0370
Epoch 8/20
3/3 [==============================] - 15s 4s/step - loss: 0.0364
Epoch 9/20
3/3 [==============================] - 14s 4s/step - loss: 0.0356
Epoch 10/20
3/3 [==============================] - 15s 4s/step - loss: 0.0346
Epoch 11/20
3/3 [==============================] - 14s 4s/step - loss: 0.0332
Epoch 12/20
3/3 [==============================] - 14s 4s/step - loss: 0.0314
Epoch 13/20
3/3 [==============================] - 14s 4s/step - loss: 0.

In [6]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
3/3 [==============================] - 38s 5s/step - loss: 0.0309
Epoch 2/20
3/3 [==============================] - 14s 4s/step - loss: 0.0309
Epoch 3/20
3/3 [==============================] - 14s 4s/step - loss: 0.0309
Epoch 4/20
3/3 [==============================] - 14s 4s/step - loss: 0.0308
Epoch 5/20
3/3 [==============================] - 14s 4s/step - loss: 0.0308
Epoch 6/20
3/3 [==============================] - 14s 4s/step - loss: 0.0308
Epoch 7/20
3/3 [==============================] - 14s 4s/step - loss: 0.0308
Epoch 8/20
3/3 [==============================] - 14s 4s/step - loss: 0.0308
Epoch 9/20
3/3 [==============================] - 14s 4s/step - loss: 0.0307
Epoch 10/20
3/3 [==============================] - 14s 4s/step - loss: 0.0307
Epoch 11/20
3/3 [==============================] - 14s 4s/step - loss: 0.0307
Epoch 12/20
3/3 [==============================] - 14s 4s/step - loss: 0.0307
Epoch 13/20
3/3 [==============================] - 14s 4s/step - loss: 0.

In [7]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
4/4 [==============================] - 42s 5s/step - loss: 0.1028
Epoch 2/20
4/4 [==============================] - 22s 5s/step - loss: 0.1022
Epoch 3/20
4/4 [==============================] - 21s 5s/step - loss: 0.1014
Epoch 4/20
4/4 [==============================] - 20s 5s/step - loss: 0.1002
Epoch 5/20
4/4 [==============================] - 20s 5s/step - loss: 0.0984
Epoch 6/20
4/4 [==============================] - 20s 5s/step - loss: 0.0955
Epoch 7/20
4/4 [==============================] - 20s 5s/step - loss: 0.0910
Epoch 8/20
4/4 [==============================] - 20s 5s/step - loss: 0.0841
Epoch 9/20
4/4 [==============================] - 20s 5s/step - loss: 0.0744
Epoch 10/20
4/4 [==============================] - 21s 5s/step - loss: 0.0629
Epoch 11/20
4/4 [==============================] - 20s 5s/step - loss: 0.0508
Epoch 12/20
4/4 [==============================] - 21s 5s/step - loss: 0.0396
Epoch 13/20
4/4 [==============================] - 20s 5s/step - loss: 0.

In [8]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
2/2 [==============================] - 32s 7s/step - loss: 0.0521
Epoch 2/20
2/2 [==============================] - 15s 7s/step - loss: 0.0520
Epoch 3/20
2/2 [==============================] - 13s 6s/step - loss: 0.0518
Epoch 4/20
2/2 [==============================] - 13s 6s/step - loss: 0.0516
Epoch 5/20
2/2 [==============================] - 13s 6s/step - loss: 0.0514
Epoch 6/20
2/2 [==============================] - 13s 6s/step - loss: 0.0512
Epoch 7/20
2/2 [==============================] - 13s 6s/step - loss: 0.0510
Epoch 8/20
2/2 [==============================] - 13s 6s/step - loss: 0.0507
Epoch 9/20
2/2 [==============================] - 13s 6s/step - loss: 0.0503
Epoch 10/20
2/2 [==============================] - 13s 6s/step - loss: 0.0498
Epoch 11/20
2/2 [==============================] - 13s 6s/step - loss: 0.0493
Epoch 12/20
2/2 [==============================] - 13s 6s/step - loss: 0.0486
Epoch 13/20
2/2 [==============================] - 13s 6s/step - loss: 0.

In [9]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
2/2 [==============================] - 30s 6s/step - loss: 0.1011
Epoch 2/20
2/2 [==============================] - 15s 6s/step - loss: 0.1009
Epoch 3/20
2/2 [==============================] - 12s 5s/step - loss: 0.1007
Epoch 4/20
2/2 [==============================] - 12s 5s/step - loss: 0.1005
Epoch 5/20
2/2 [==============================] - 12s 5s/step - loss: 0.1004
Epoch 6/20
2/2 [==============================] - 12s 5s/step - loss: 0.1002
Epoch 7/20
2/2 [==============================] - 12s 5s/step - loss: 0.1001
Epoch 8/20
2/2 [==============================] - 12s 5s/step - loss: 0.0999
Epoch 9/20
2/2 [==============================] - 12s 5s/step - loss: 0.0997
Epoch 10/20
2/2 [==============================] - 11s 5s/step - loss: 0.0994
Epoch 11/20
2/2 [==============================] - 11s 5s/step - loss: 0.0992
Epoch 12/20
2/2 [==============================] - 11s 5s/step - loss: 0.0988
Epoch 13/20
2/2 [==============================] - 12s 5s/step - loss: 0.

In [10]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
3/3 [==============================] - 36s 5s/step - loss: 0.1226
Epoch 2/20
3/3 [==============================] - 16s 4s/step - loss: 0.1224
Epoch 3/20
3/3 [==============================] - 15s 4s/step - loss: 0.1221
Epoch 4/20
3/3 [==============================] - 14s 4s/step - loss: 0.1218
Epoch 5/20
3/3 [==============================] - 14s 4s/step - loss: 0.1215
Epoch 6/20
3/3 [==============================] - 14s 4s/step - loss: 0.1210
Epoch 7/20
3/3 [==============================] - 14s 4s/step - loss: 0.1204
Epoch 8/20
3/3 [==============================] - 14s 4s/step - loss: 0.1196
Epoch 9/20
3/3 [==============================] - 14s 4s/step - loss: 0.1185
Epoch 10/20
3/3 [==============================] - 14s 4s/step - loss: 0.1170
Epoch 11/20
3/3 [==============================] - 14s 4s/step - loss: 0.1151
Epoch 12/20
3/3 [==============================] - 14s 4s/step - loss: 0.1127
Epoch 13/20
3/3 [==============================] - 14s 4s/step - loss: 0.

In [11]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
3/3 [==============================] - 36s 5s/step - loss: 0.0679
Epoch 2/20
3/3 [==============================] - 17s 5s/step - loss: 0.0676
Epoch 3/20
3/3 [==============================] - 17s 5s/step - loss: 0.0674
Epoch 4/20
3/3 [==============================] - 16s 5s/step - loss: 0.0671
Epoch 5/20
3/3 [==============================] - 16s 5s/step - loss: 0.0668
Epoch 6/20
3/3 [==============================] - 16s 5s/step - loss: 0.0663
Epoch 7/20
3/3 [==============================] - 16s 5s/step - loss: 0.0657
Epoch 8/20
3/3 [==============================] - 16s 5s/step - loss: 0.0649
Epoch 9/20
3/3 [==============================] - 16s 5s/step - loss: 0.0636
Epoch 10/20
3/3 [==============================] - 16s 5s/step - loss: 0.0619
Epoch 11/20
3/3 [==============================] - 16s 5s/step - loss: 0.0595
Epoch 12/20
3/3 [==============================] - 16s 5s/step - loss: 0.0557
Epoch 13/20
3/3 [==============================] - 16s 5s/step - loss: 0.

In [12]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
2/2 [==============================] - 30s 6s/step - loss: 0.0183
Epoch 2/20
2/2 [==============================] - 13s 6s/step - loss: 0.0183
Epoch 3/20
2/2 [==============================] - 13s 6s/step - loss: 0.0183
Epoch 4/20
2/2 [==============================] - 12s 5s/step - loss: 0.0183
Epoch 5/20
2/2 [==============================] - 12s 5s/step - loss: 0.0182
Epoch 6/20
2/2 [==============================] - 13s 6s/step - loss: 0.0182
Epoch 7/20
2/2 [==============================] - 12s 5s/step - loss: 0.0182
Epoch 8/20
2/2 [==============================] - 12s 5s/step - loss: 0.0182
Epoch 9/20
2/2 [==============================] - 12s 5s/step - loss: 0.0182
Epoch 10/20
2/2 [==============================] - 12s 5s/step - loss: 0.0182
Epoch 11/20
2/2 [==============================] - 12s 5s/step - loss: 0.0181
Epoch 12/20
2/2 [==============================] - 12s 5s/step - loss: 0.0181
Epoch 13/20
2/2 [==============================] - 12s 5s/step - loss: 0.

In [13]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
1/1 [==============================] - 17s 17s/step - loss: 0.1391
Epoch 2/20
1/1 [==============================] - 3s 3s/step - loss: 0.1390
Epoch 3/20
1/1 [==============================] - 3s 3s/step - loss: 0.1389
Epoch 4/20
1/1 [==============================] - 3s 3s/step - loss: 0.1388
Epoch 5/20
1/1 [==============================] - 3s 3s/step - loss: 0.1387
Epoch 6/20
1/1 [==============================] - 3s 3s/step - loss: 0.1386
Epoch 7/20
1/1 [==============================] - 3s 3s/step - loss: 0.1385
Epoch 8/20
1/1 [==============================] - 3s 3s/step - loss: 0.1383
Epoch 9/20
1/1 [==============================] - 3s 3s/step - loss: 0.1382
Epoch 10/20
1/1 [==============================] - 3s 3s/step - loss: 0.1381
Epoch 11/20
1/1 [==============================] - 3s 3s/step - loss: 0.1380
Epoch 12/20
1/1 [==============================] - 3s 3s/step - loss: 0.1378
Epoch 13/20
1/1 [==============================] - 4s 4s/step - loss: 0.1377
Epoch 

In [14]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
2/2 [==============================] - 31s 5s/step - loss: 0.0686
Epoch 2/20
2/2 [==============================] - 14s 5s/step - loss: 0.0684
Epoch 3/20
2/2 [==============================] - 12s 5s/step - loss: 0.0682
Epoch 4/20
2/2 [==============================] - 11s 5s/step - loss: 0.0680
Epoch 5/20
2/2 [==============================] - 11s 5s/step - loss: 0.0678
Epoch 6/20
2/2 [==============================] - 11s 5s/step - loss: 0.0675
Epoch 7/20
2/2 [==============================] - 11s 4s/step - loss: 0.0672
Epoch 8/20
2/2 [==============================] - 11s 4s/step - loss: 0.0667
Epoch 9/20
2/2 [==============================] - 11s 5s/step - loss: 0.0662
Epoch 10/20
2/2 [==============================] - 11s 5s/step - loss: 0.0655
Epoch 11/20
2/2 [==============================] - 11s 4s/step - loss: 0.0647
Epoch 12/20
2/2 [==============================] - 11s 5s/step - loss: 0.0635
Epoch 13/20
2/2 [==============================] - 11s 4s/step - loss: 0.

In [15]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
2/2 [==============================] - 34s 9s/step - loss: 0.0216
Epoch 2/20
2/2 [==============================] - 14s 7s/step - loss: 0.0215
Epoch 3/20
2/2 [==============================] - 13s 6s/step - loss: 0.0213
Epoch 4/20
2/2 [==============================] - 13s 6s/step - loss: 0.0212
Epoch 5/20
2/2 [==============================] - 14s 7s/step - loss: 0.0210
Epoch 6/20
2/2 [==============================] - 13s 6s/step - loss: 0.0207
Epoch 7/20
2/2 [==============================] - 13s 6s/step - loss: 0.0205
Epoch 8/20
2/2 [==============================] - 13s 6s/step - loss: 0.0201
Epoch 9/20
2/2 [==============================] - 13s 6s/step - loss: 0.0197
Epoch 10/20
2/2 [==============================] - 13s 6s/step - loss: 0.0192
Epoch 11/20
2/2 [==============================] - 14s 7s/step - loss: 0.0186
Epoch 12/20
2/2 [==============================] - 14s 7s/step - loss: 0.0178
Epoch 13/20
2/2 [==============================] - 13s 6s/step - loss: 0.

In [16]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
2/2 [==============================] - 31s 6s/step - loss: 0.1123
Epoch 2/20
2/2 [==============================] - 13s 6s/step - loss: 0.1122
Epoch 3/20
2/2 [==============================] - 12s 6s/step - loss: 0.1121
Epoch 4/20
2/2 [==============================] - 12s 6s/step - loss: 0.1121
Epoch 5/20
2/2 [==============================] - 12s 6s/step - loss: 0.1120
Epoch 6/20
2/2 [==============================] - 12s 6s/step - loss: 0.1119
Epoch 7/20
2/2 [==============================] - 13s 6s/step - loss: 0.1117
Epoch 8/20
2/2 [==============================] - 12s 6s/step - loss: 0.1116
Epoch 9/20
2/2 [==============================] - 13s 6s/step - loss: 0.1115
Epoch 10/20
2/2 [==============================] - 12s 6s/step - loss: 0.1112
Epoch 11/20
2/2 [==============================] - 12s 6s/step - loss: 0.1110
Epoch 12/20
2/2 [==============================] - 12s 6s/step - loss: 0.1107
Epoch 13/20
2/2 [==============================] - 12s 6s/step - loss: 0.

In [17]:
df = pd.DataFrame(res)
df.to_csv('DCAEOCSVM_SV.csv')